In [23]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-ec54fbe500fc3b5c.parquet', 'validation': 'data/validation-00000-of-00001-3cf888b12fff1dd6.parquet'}
train_df = pd.read_parquet("hf://datasets/lucadiliello/newsqa/" + splits["train"])

In [24]:
train_df

,context,question,answers,key,labels
0,"NEW DELHI, India (CNN) -- A high court in nort...",What was the amount of children murdered?,[19],da0e6b66e04d439fa1ba23c32de07e50,"[{'end': [295], 'start': [294]}]"
1,"NEW DELHI, India (CNN) -- A high court in nort...",When was Pandher sentenced to death?,[February.],724f6eb9a2814e4fb2d7d8e4de846073,"[{'end': [269], 'start': [261]}]"
2,"NEW DELHI, India (CNN) -- A high court in nort...",The court aquitted Moninder Singh Pandher of w...,[rape and murder],d64cbb90e5134081acfa83d3e702408c,"[{'end': [638], 'start': [624]}]"
3,"NEW DELHI, India (CNN) -- A high court in nort...",who was acquitted,[Moninder Singh Pandher],fd7177ee6f1f4d62becd983a0305f503,"[{'end': [216], 'start': [195]}]"
4,"NEW DELHI, India (CNN) -- A high court in nort...",who was sentenced,[Moninder Singh Pandher],cd25c69f631349748ccdeccaace66463,"[{'end': [216], 'start': [195]}]"
...,...,...,...,...,...
74155,"OAKLAND, California (CNN) -- Fifth-grader Chri...",What happened to Christopher Rodriguez?,"[was hit by a stray bullet, paralyzing him for...",c0ac3ef6afb94dbe8e1666b8bfbf5237,"[{'end': [259], 'start': [209]}]"
74156,"OAKLAND, California (CNN) -- Fifth-grader Chri...",What did Christopher Rodriguez love?,[music.],683cfaf6ec1c47189172300b4aaa3f91,"[{'end': [985], 'start': [980]}]"
74157,"OAKLAND, California (CNN) -- Fifth-grader Chri...",WIll the boy walk again?,[paralyzed for life],0da315692fd04023b7205fab7aeeb26e,"[{'end': [701], 'start': [684]}]"
74158,"OAKLAND, California (CNN) -- Fifth-grader Chri...",What did the suspect allegedly rob?,[Chevron gas station],74a4fa2d8548463f8fdf9a370d7ea5ff,"[{'end': [423], 'start': [405]}]"


In [25]:
!pip install gensim

In [26]:
import gensim
import torch
import torchvision.transforms
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gensim.models import Word2Vec
torch.manual_seed(1)
import numpy as np
import re

device="cuda" if torch.cuda.is_available() else "cpu"

In [62]:
text=[]
tokens=[]

for i in range (len(train_df)):
  sentence=str(train_df["context"][i]+" "+train_df["question"][i]+" "+train_df["answers"][i])
  tokens=re.findall(r'\b\w+\b',sentence)
  text.append(tokens)


In [28]:
# Using the Word2Vec architecture
model_0=Word2Vec(sentences=text, vector_size=10, window=5, min_count=1, workers=4)



In [50]:
word_vectors=model_0.wv
weights=torch.FloatTensor(word_vectors.vectors)

embedding_layer=nn.Embedding.from_pretrained(weights)
word_to_idx=word_vectors.key_to_index


In [51]:
embedding_layer.weight[2]

tensor([ 0.1241, -0.2322, -0.1147,  2.5470,  5.7591, -0.2311, -6.2227, -5.3876,
         3.9297,  0.6262])

In [55]:
data=[]
for word,idx in word_to_idx.items():
  embedding=embedding_layer.weight[idx]
  data.append({"word":word,"embedding":embedding.tolist()})


In [56]:
Word2Vec_df=pd.DataFrame(data,columns=["word","embedding"])

In [59]:
Word2Vec_df.head(20)

,word,embedding
0,n,"[3.1490302085876465, 0.2023921012878418, -1.21..."
1,the,"[-2.157212257385254, -0.9072088599205017, -3.1..."
2,to,"[0.12405329942703247, -0.23224273324012756, -0..."
3,of,"[-4.464725971221924, 0.48568210005760193, -4.2..."
4,a,"[-3.0251123905181885, 2.1898245811462402, -3.2..."
5,and,"[-3.8439362049102783, 2.1528327465057373, -2.8..."
6,in,"[-3.4177918434143066, -0.30659985542297363, -6..."
7,s,"[-0.6604267954826355, 2.9722378253936768, -2.2..."
8,said,"[6.367203235626221, 1.7926098108291626, -1.665..."
9,that,"[0.7505955100059509, 2.634713649749756, -2.351..."


In [60]:
Word2Vec_df.to_csv('Word2Vec.csv.gz',index=False,compression="gzip")

## FastText architecture

In [61]:
from gensim.models import FastText

In [63]:
model_1=FastText(sentences=text,window=5,min_count=5,workers=8, epochs=2,vector_size=10,min_n=5)


In [64]:
word_vectors2=model_1.wv
weights2=torch.FloatTensor(word_vectors2.vectors)

embedding_layer_2=nn.Embedding.from_pretrained(weights2)


In [70]:
word_to_idx2=word_vectors2.key_to_index
data=[]
for word,idx in word_to_idx2.items():
  embedding=embedding_layer_2.weight[idx]
  data.append({"word":word,"embedding":embedding.tolist()})

In [71]:
FastText_df=pd.DataFrame(data,columns=["word","embedding"])

In [72]:
FastText_df.head()

,word,embedding
0,n,"[7.555756092071533, -12.777887344360352, 4.670..."
1,the,"[-3.104872703552246, -1.0326917171478271, -10...."
2,to,"[-12.41839599609375, 19.416269302368164, -12.7..."
3,of,"[6.122688293457031, -0.8311682343482971, -20.0..."
4,a,"[-2.7925400733947754, 4.5804548263549805, -14...."


In [74]:
FastText_df.to_csv("FastText.csv.gz",index=False,compression="gzip")

## Transformer architecture for embeddings